In [1]:
!pip install transformers[torch] datasets accelerate sentencepiece optimum auto-gptq bitsandbytes peft

Looking in indexes: http://pypi-cache/index, https://pypi.ngc.nvidia.com
     |████████████████████████████████| 8.3 MB 728 kB/s eta 0:00:01
     |████████████████████████████████| 521 kB 159.1 MB/s eta 0:00:01
     |████████████████████████████████| 265 kB 160.8 MB/s eta 0:00:01
     |████████████████████████████████| 1.3 MB 180.5 MB/s eta 0:00:01
     |████████████████████████████████| 403 kB 103.7 MB/s eta 0:00:01
     |████████████████████████████████| 4.8 MB 2.8 MB/s eta 0:00:01
     |████████████████████████████████| 92.6 MB 4.5 MB/s eta 0:00:01
     |████████████████████████████████| 168 kB 113.3 MB/s eta 0:00:01
     |████████████████████████████████| 1.3 MB 134.1 MB/s eta 0:00:01
     |████████████████████████████████| 3.8 MB 111.2 MB/s eta 0:00:01
     |████████████████████████████████| 311 kB 154.0 MB/s eta 0:00:01
     |████████████████████████████████| 115 kB 165.7 MB/s eta 0:00:01
     |████████████████████████████████| 194 kB 182.2 MB/s eta 0:00:01
     |████████████████

In [47]:
pip install wandb

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


Looking in indexes: http://pypi-cache/index, https://pypi.ngc.nvidia.com
     |████████████████████████████████| 2.1 MB 173.2 MB/s eta 0:00:01
     |████████████████████████████████| 190 kB 162.8 MB/s eta 0:00:01
     |████████████████████████████████| 254 kB 92.5 MB/s eta 0:00:01
     |████████████████████████████████| 62 kB 99.7 MB/s  eta 0:00:01
You should consider upgrading via the '/bin/python -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.


In [1]:
import os
import json

import torch


In [2]:
import os
split = "dev.json"
data = json.load(open(f"Complete_dataset/{split}"))
files = os.listdir("Complete_dataset/CT json/")
files.remove(".DS_Store")

In [3]:
files_data = {file[:-5]:json.load(open(f"Complete_dataset/CT json/{file}")) for file in files}

In [4]:
data_expanded = []
for _id, value in data.items():
    temp = {}
    temp["id"] = _id
    p_nctid = value["Primary_id"]
    s_nctid = value.get("Secondary_id")
    section_id = value["Section_id"]
    statement = value["Statement"]
    primary_evidence = files_data[p_nctid][section_id]
    temp["statement"] = statement
    temp["primary_evidence"] = primary_evidence
    temp["label"] = value["Label"]
    
    if s_nctid is not None:
        secondary_evidence = files_data[s_nctid][section_id]
        temp["secondary_evidence"] = secondary_evidence
    
    data_expanded.append(temp)

In [5]:
from tqdm import tqdm
from sklearn.metrics import (
    accuracy_score,
    f1_score,
    precision_score,
    recall_score,
)
def compute_metrics_decoded(decoded_labs, decoded_preds):

    metrics = {
        "micro_f1": f1_score(decoded_labs, decoded_preds, average="micro"),
        "macro_f1": f1_score(decoded_labs, decoded_preds, average="macro"),
        "precision": precision_score(decoded_labs, decoded_preds, average="micro"),
        "recall": recall_score(decoded_labs, decoded_preds, average="micro"),
        "accuracy": accuracy_score(decoded_labs, decoded_preds),
    }

    return metrics


def get_input_text_p(premise, hypothesis, PROMPT):
    return PROMPT.format(premise=premise, hypothesis=hypothesis)


Mistral-v01

In [6]:
from transformers import AutoTokenizer, AutoModelForCausalLM
from transformers import LlamaTokenizer, LlamaForCausalLM, pipeline

tokenizer = AutoTokenizer.from_pretrained("models/Mistral-7B-v0.1")
tokenizer.pad_token = tokenizer.eos_token  # Most LLMs don't have a pad token by default
model = AutoModelForCausalLM.from_pretrained(
    "models/Mistral-7B-v0.1", device_map="auto",# load_in_4bit=True
)



Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [8]:
pipe = pipeline(
    task='text-generation',
    model=model,
    tokenizer=tokenizer,
    #max_new_tokens=64,
    #max_length=2048,
    do_sample=True,
    temperature=0.7,
    top_p=0.95,
    top_k=40,
    repetition_penalty=1.1,
  
)

In [74]:

PROMPT = """
a = {premise} 
b = '{hypothesis}'. 
Statement: a implies b. Is it True or False ?
###Answer:
"""


samples = []
for sample in data_expanded:
    primary_evidence = "".join(sample['primary_evidence'])
    sentence = f"Primary trial evidence are {primary_evidence}"
    secondary_evidence = sample.get("secondary_evidence")
    if secondary_evidence:
        secondary_evidence = "".join(sample['secondary_evidence'])
        sentence = f"{sentence} Secondary trial evidence are {secondary_evidence}"
    input_text = get_input_text_p(sentence, sample['statement'], PROMPT)
    temp = {"text":input_text, "label":sample['label']}
    samples.append(temp)

In [75]:
input_ids = tokenizer(
            samples[2]['text'], return_tensors="pt"
        ).input_ids.cuda()

output = pipe(samples[2]['text'], max_length=input_ids.shape[1]+8)

output[0]['generated_text']

/usr/local/lib/python3.10/dist-packages/transformers/pipelines/base.py:1101: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


"\na = Primary trial evidence are Adverse Events 1:  Total: 20/167 (11.98%)  Cardiac failure acute 0/167 (0.00%)  Diarrhoea 5/167 (2.99%)  Colitis 2/167 (1.20%)  Enterocolitis 1/167 (0.60%)  Enterocolitis haemorrhagic 1/167 (0.60%)  Stomatitis 1/167 (0.60%)  Impaired healing 1/167 (0.60%)  Sudden death 1/167 (0.60%)  Postoperative wound infection 2/167 (1.20%)  Erysipelas 2/167 (1.20%)  Bacterial diarrhoea 1/167 (0.60%)Adverse Events 2:  Total: 4/167 (2.40%)  Cardiac failure acute 1/167 (0.60%)  Diarrhoea 0/167 (0.00%)  Colitis 0/167 (0.00%)  Enterocolitis 0/167 (0.00%)  Enterocolitis haemorrhagic 0/167 (0.00%)  Stomatitis 0/167 (0.00%)  Impaired healing 0/167 (0.00%)  Sudden death 0/167 (0.00%)  Postoperative wound infection 1/167 (0.60%)  Erysipelas 0/167 (0.00%)  Bacterial diarrhoea 0/167 (0.00%) Secondary trial evidence are Adverse Events 1:  Total: 112/458 (24.45%)  Febrile neutropenia 8/458 (1.75%)  Neutropenia 6/458 (1.31%)  Anaemia 3/458 (0.66%)  Thrombocytopenia 3/458 (0.66%) 

In [135]:
res[3]

"Primary trial evidence are Outcome Measurement:   Local Control Using Ipsilateral Breast Tumor Recurrence Rates  [Not Specified]  Time frame: 2 years after treatment completionResults 1:   Arm/Group Title: Accelerated Partial Breast Brachytherapy  Arm/Group Description: Each patient will receive accelerated partial breast brachytherapy with multiple plane implant.  Patients will receive 3400 cGy delivered in 10 twice-daily fractions. Treatment is to be given over 5-7 days with a minimum of 6 hours separation between fractions.  Overall Number of Participants Analyzed: 151  Measure Type: Number  Unit of Measure: percentage of participants  .7\n###Question: Does this imply that the primary trial does not report the PFS or objective response rate of its patient cohort ? If yes then answer 'Entailment' and if no then answer 'Contradiction'\n###Answer:\n###Justification:"

In [7]:
# BEST


PROMPT = """{premise}
Question: Does this imply that '{hypothesis}' ? Answer Yes or No.
### Answer:
"""

PROMPT2 = """<s>[INST]{premise}
###Question: Does this imply that {hypothesis} ? Answer Yes or No.[/INST]
###Answer:
"""

PROMPT3 = """{premise}
###Question: Does this imply that {hypothesis} ? If yes then answer 'Entailment' and if no then answer 'Contradiction'
###Answer:
"""

PROMPT4 = """ Complete the sentence using given options:
Given Premise: '{premise}' and Hypothesis: '{hypothesis}' are 
Options:
Entailing
Contradicting
###Answer:
"""

PROMPT5 = """
a = {premise} 
b = '{hypothesis}'. 
Statement: a implies b. Is it True or False ?
###Answer:
"""

samples = []
for sample in data_expanded:
    primary_evidence = "".join(sample['primary_evidence'])
    sentence = f"Primary trial evidence are {primary_evidence}"
    secondary_evidence = sample.get("secondary_evidence")
    if secondary_evidence:
        secondary_evidence = "".join(sample['secondary_evidence'])
        sentence = f"{sentence} Secondary trial evidence are {secondary_evidence}"
    input_text = get_input_text_p(sentence, sample['statement'], PROMPT)
    if 'label' in sample:
        temp = {"text":input_text, "label":sample['label'], "id": sample['id']}
    else:
        temp = {"text":input_text, "id": sample['id']}
    samples.append(temp)




In [8]:
sample = samples[4]['text']

inputs = tokenizer(sample, return_tensors="pt")
model.eval()
with torch.no_grad():
    outputs = model.generate(
        input_ids=inputs["input_ids"].to("cuda"), 
        max_new_tokens=1,#len(inputs["input_ids"])+8,
        do_sample=True,
        temperature=0.7,
        top_p=0.95,
        top_k=40,
        repetition_penalty=1.1,
        output_scores=True, return_dict_in_generate=True
        )
    #print(tokenizer.batch_decode(outputs.detach().cpu().numpy(), skip_special_tokens=True)[0])

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


tensor([[28733]], device='cuda:0')

In [9]:
import numpy as np
transition_scores = model.compute_transition_scores(
    outputs.sequences, outputs.scores, normalize_logits=True
)
# input_length is the length of the input prompt for decoder-only models, like the GPT family, and 1 for
# encoder-decoder models, like BART or T5.
input_length = 1 if model.config.is_encoder_decoder else inputs.input_ids.shape[1]
generated_tokens = outputs.sequences[:, input_length:]
for tok, score in zip(generated_tokens[0], transition_scores[0]):
    # | token | token string | logits | probability
    print(f"| {tok:5d} | {tokenizer.decode(tok):8s} | {score.cpu().numpy():.3f} | {np.exp(score.cpu().numpy()):.2%}")



|  5613 | Yes      | -0.752 | 47.12%


In [11]:
from collections import defaultdict as dd
from tqdm import tqdm
import numpy as np
results = dd(lambda: dd(lambda: dd(lambda: {'count': 0, 'score': []})))

k=0
while (k < 5):
    res = dd()
    labels = []
    for sample in tqdm(samples):
        id_ = sample['id']
        labels.append(sample['label'])
        inputs = tokenizer(sample['text'], return_tensors="pt")
        with torch.inference_mode():
            outputs = model.generate(
            input_ids=inputs["input_ids"].to("cuda"), 
            max_new_tokens=1,#len(inputs["input_ids"])+8,
            do_sample=True,
            temperature=0.7,
            top_p=0.95,
            top_k=40,
            repetition_penalty=1.1,
            output_scores=True, return_dict_in_generate=True,
            )
            transition_scores = model.compute_transition_scores(
            outputs.sequences, outputs.scores, normalize_logits=True
            )
            # input_length is the length of the input prompt for decoder-only models, like the GPT family, and 1 for
            # encoder-decoder models, like BART or T5.
            input_length = 1 if model.config.is_encoder_decoder else inputs.input_ids.shape[1]
            generated_tokens = outputs.sequences[:, input_length:]
            res[id_] = {"score": float(np.exp(transition_scores.cpu())), "output": tokenizer.decode(generated_tokens[0])} #tokenizer.batch_decode(outputs.detach().cpu().numpy(), skip_special_tokens=True)[0]



    pred = []

    for id_, r in res.items():
        word = r['output']
        if 'yes' in word.lower():
            results[id_]['prediction']['Entailment']['count'] += 1 
            results[id_]['prediction']['Entailment']['score'].append(r['score']) 
            #pred.append('Entailment')
            #labs.append(labels[i])
        elif 'no' in word.lower():
            results[id_]['prediction']['Contradiction']['count'] += 1
            results[id_]['prediction']['Contradiction']['score'].append(r['score']) 
    
    k += 1

  0%|          | 0/200 [00:00<?, ?it/s]The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
  0%|          | 1/200 [00:00<00:29,  6.77it/s]The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
  1%|          | 2/200 [00:00<01:02,  3.16it/s]The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
  2%|▏         | 3/200 [00:00<01:08,  2.89it/s]The attention mask and the pad token id were not set. As a consequence, you may

In [12]:
labels = dd(str)
for sample in tqdm(samples):
    id_ = sample['id']
    labels[id_] = sample['label']

def get_predicted_label(instance_data):
    # Get the 'prediction' sub-dictionary
    prediction_data = instance_data.get('prediction', {})

    # Initialize variables to store the predicted label and its count
    predicted_label = None
    max_count = 0

    # Iterate through the prediction data to find the label with the highest count
    for label, label_data in prediction_data.items():
        count = label_data.get('count', 0)
        if count > max_count:
            max_count = count
            predicted_label = label

    return predicted_label

# Create a list to store true labels and predicted labels
true_labels = []
predicted_labels = []

# Iterate through your data
for instance_id, instance_data in results.items():
    gold_label = labels[instance_id]  # You need to implement a function to get the gold label based on instance_id
    true_labels.append(gold_label)

    predicted_label = get_predicted_label(instance_data)  # You need to implement a function to get the predicted label based on instance_data
    predicted_labels.append(predicted_label)


100%|██████████| 200/200 [00:00<00:00, 1122972.96it/s]

               precision    recall  f1-score   support

Contradiction       0.47      0.29      0.35        98
   Entailment       0.49      0.68      0.57       100

     accuracy                           0.48       198
    macro avg       0.48      0.48      0.46       198
 weighted avg       0.48      0.48      0.46       198



In [13]:

from sklearn.metrics import classification_report

print(classification_report(true_labels, predicted_labels))

               precision    recall  f1-score   support

Contradiction       0.47      0.29      0.35        98
   Entailment       0.49      0.68      0.57       100

     accuracy                           0.48       198
    macro avg       0.48      0.48      0.46       198
 weighted avg       0.48      0.48      0.46       198



In [83]:
from sklearn.metrics import classification_report

print(classification_report(labs, pred))

               precision    recall  f1-score   support

Contradiction       0.56      0.29      0.39        92
   Entailment       0.53      0.78      0.63        95

     accuracy                           0.54       187
    macro avg       0.55      0.54      0.51       187
 weighted avg       0.55      0.54      0.51       187



Mistral-Instruct-v2

In [6]:
from transformers import AutoTokenizer, AutoModelForCausalLM
from transformers import LlamaTokenizer, LlamaForCausalLM, pipeline

tokenizer = AutoTokenizer.from_pretrained("models/llama-2-13b")
tokenizer.pad_token = tokenizer.eos_token  # Most LLMs don't have a pad token by default
model = AutoModelForCausalLM.from_pretrained(
    "models/llama-2-13b", device_map="auto",# load_in_4bit=True
)



Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

/usr/local/lib/python3.10/dist-packages/torch/_utils.py:783: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()


In [7]:
# BEST

PROMPT_B = """{premise}
Question: Does this imply that '{hypothesis}' ? Answer Yes or No.
### Answer:
"""


PROMPT = """<s>[INST]{premise}
###Question: Does this imply that {hypothesis} ? Answer Yes or No.[/INST]
###Answer:
"""
text = "<s>[INST] What is your favourite condiment? [/INST]"
"Well, I'm quite partial to a good squeeze of fresh lemon juice. It adds just the right amount of zesty flavour to whatever I'm cooking up in the kitchen!</s> "
"[INST] Do you have mayonnaise recipes? [/INST]"


PROMPT_1 = """<s>[INST]Context
Premise: {premise}
Hypothesis: {hypothesis}
If Premise entails Hypothesis then return Entailment otherwise return Contradiction[/INST]
Output: 
"""

INFER_PROMPT = """
<s>### Instruction:
Read the input text and answer the following question with Yes or No.

### Input:
{premise}

Question: Does this imply that {hypothesis}?

### Response:
</s>
"""

INFER_PROMPT_2 = """
<s>### Instruction:

Claim: {hypothesis}

Read the following text carefully to decide if given Claim is True or False. Make sure to give an answer. Be decisive.

{premise}

### Response:
</s>
"""

samples = []
for sample in data_expanded:
    primary_evidence = "".join(sample['primary_evidence'])
    sentence = f"Primary trial evidence are {primary_evidence}"
    secondary_evidence = sample.get("secondary_evidence")
    if secondary_evidence:
        secondary_evidence = "".join(sample['secondary_evidence'])
        sentence = f"{sentence} Secondary trial evidence are {secondary_evidence}"
    input_text = get_input_text_p(sentence, sample['statement'], PROMPT_B)
    if 'label' in sample:
        temp = {"text":input_text, "label":sample['label'], "id": sample['id']}
    else:
        temp = {"text":input_text, "id": sample['id']}
    samples.append(temp)




In [8]:
sample = samples[10]['text']

inputs = tokenizer(sample, return_tensors="pt")
model.eval()
with torch.no_grad():
    outputs = model.generate(
        input_ids=inputs["input_ids"].to("cuda"), 
        max_new_tokens=1,#len(inputs["input_ids"])+8,
        do_sample=True,
        temperature=0.7,
        top_p=0.95,
        top_k=40,
        repetition_penalty=1.1,
        output_scores=True, return_dict_in_generate=True
        )
    #print(tokenizer.batch_decode(outputs.detach().cpu().numpy(), skip_special_tokens=True)[0])

import numpy as np
transition_scores = model.compute_transition_scores(
    outputs.sequences, outputs.scores, normalize_logits=True
)
# input_length is the length of the input prompt for decoder-only models, like the GPT family, and 1 for
# encoder-decoder models, like BART or T5.
input_length = 1 if model.config.is_encoder_decoder else inputs.input_ids.shape[1]
generated_tokens = outputs.sequences[:, input_length:]
for tok, score in zip(generated_tokens[0], transition_scores[0]):
    # | token | token string | logits | probability
    print(f"| {tok:5d} | {tokenizer.decode(tok):8s} | {score.cpu().numpy():.3f} | {np.exp(score.cpu().numpy()):.2%}")

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


|  3782 | No       | -1.518 | 21.92%


In [9]:
from collections import defaultdict as dd
from tqdm import tqdm
import numpy as np
results = dd(lambda: dd(lambda: dd(lambda: {'count': 0, 'score': []})))

k=0
while (k < 5):
    res = dd()
    labels = []
    for sample in tqdm(samples):
        id_ = sample['id']
        labels.append(sample['label'])
        inputs = tokenizer(sample['text'], return_tensors="pt")
        with torch.inference_mode():
            outputs = model.generate(
            input_ids=inputs["input_ids"].to("cuda"), 
            max_new_tokens=1,#len(inputs["input_ids"])+8,
            do_sample=True,
            temperature=0.7,
            top_p=0.95,
            top_k=40,
            repetition_penalty=1.1,
            output_scores=True, return_dict_in_generate=True,
            )
            transition_scores = model.compute_transition_scores(
            outputs.sequences, outputs.scores, normalize_logits=True
            )
            # input_length is the length of the input prompt for decoder-only models, like the GPT family, and 1 for
            # encoder-decoder models, like BART or T5.
            input_length = 1 if model.config.is_encoder_decoder else inputs.input_ids.shape[1]
            generated_tokens = outputs.sequences[:, input_length:]
            res[id_] = {"score": float(np.exp(transition_scores.cpu())), "output": tokenizer.decode(generated_tokens[0])} #tokenizer.batch_decode(outputs.detach().cpu().numpy(), skip_special_tokens=True)[0]


    pred = []

    for id_, r in res.items():
        word = r['output']
        if 'yes' in word.lower():
            results[id_]['prediction']['Entailment']['count'] += 1 
            results[id_]['prediction']['Entailment']['score'].append(r['score']) 
            #pred.append('Entailment')
            #labs.append(labels[i])
        elif 'no' in word.lower():
            results[id_]['prediction']['Contradiction']['count'] += 1
            results[id_]['prediction']['Contradiction']['score'].append(r['score']) 
    
    k += 1

  0%|          | 0/200 [00:00<?, ?it/s]The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
  0%|          | 1/200 [00:00<00:26,  7.54it/s]The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
  1%|          | 2/200 [00:00<00:57,  3.47it/s]The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
  2%|▏         | 3/200 [00:00<00:57,  3.46it/s]The attention mask and the pad token id were not set. As a consequence, you may

In [10]:
labels = dd(str)
for sample in tqdm(samples):
    id_ = sample['id']
    labels[id_] = sample['label']

def get_predicted_label(instance_data):
    # Get the 'prediction' sub-dictionary
    prediction_data = instance_data.get('prediction', {})

    # Initialize variables to store the predicted label and its count
    predicted_label = None
    max_count = 0

    # Iterate through the prediction data to find the label with the highest count
    for label, label_data in prediction_data.items():
        count = label_data.get('count', 0)
        if count > max_count:
            max_count = count
            predicted_label = label

    return predicted_label

# Create a list to store true labels and predicted labels
true_labels = []
predicted_labels = []

# Iterate through your data
for instance_id, instance_data in results.items():
    gold_label = labels[instance_id]  # You need to implement a function to get the gold label based on instance_id
    true_labels.append(gold_label)

    predicted_label = get_predicted_label(instance_data)  # You need to implement a function to get the predicted label based on instance_data
    predicted_labels.append(predicted_label)


100%|██████████| 200/200 [00:00<00:00, 574168.93it/s]


In [11]:

from sklearn.metrics import classification_report

print(classification_report(true_labels, predicted_labels))

               precision    recall  f1-score   support

Contradiction       0.60      0.54      0.57        92
   Entailment       0.57      0.62      0.59        89

     accuracy                           0.58       181
    macro avg       0.58      0.58      0.58       181
 weighted avg       0.58      0.58      0.58       181



In [19]:
sample = samples[2]['text']

inputs = tokenizer(sample, return_tensors="pt")
model.eval()
with torch.no_grad():
    outputs = model.generate(
        input_ids=inputs["input_ids"].to("cuda"), 
        max_new_tokens=2,#len(inputs["input_ids"])+16,
        do_sample=True,
        temperature=0.7,
        top_p=0.95,
        top_k=40,
        repetition_penalty=1.1,
        #output_scores=True, return_dict_in_generate=True
        )
    print(tokenizer.batch_decode(outputs.detach().cpu().numpy(), skip_special_tokens=True)[0])

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


[INST]Primary trial evidence are Adverse Events 1:  Total: 5/32 (15.63%)  Febrile neutropenia 1/32 (3.13%)  Supraventricular tachycardia 1/32 (3.13%)  Hypersensitivity 2/32 (6.25%)  Catheter site infection 1/32 (3.13%)  Confusional state 1/32 (3.13%) Secondary trial evidence are Adverse Events 1:  Total: 285/752 (37.90%)  Anaemia 2/752 (0.27%)  Disseminated intravascular coagulation 2/752 (0.27%)  Febrile neutropenia 51/752 (6.78%)  Neutropenia 47/752 (6.25%)  Thrombocytopenia 2/752 (0.27%)  Atrial fibrillation 1/752 (0.13%)  Atrial flutter 0/752 (0.00%)  Cardiac failure congestive 1/752 (0.13%)  Left ventricular dysfunction 0/752 (0.00%)Adverse Events 2:  Total: 117/382 (30.63%)  Anaemia 3/382 (0.79%)  Disseminated intravascular coagulation 0/382 (0.00%)  Febrile neutropenia 11/382 (2.88%)  Neutropenia 20/382 (5.24%)  Thrombocytopenia 0/382 (0.00%)  Atrial fibrillation 1/382 (0.26%)  Atrial flutter 1/382 (0.26%)  Cardiac failure congestive 0/382 (0.00%)  Left ventricular dysfunction 1

In [ ]:
res = []
labels = []
for sample in tqdm(samples):
    labels.append(sample['label'])
    input_ids = tokenizer(
            sample['text'], return_tensors="pt"
        ).input_ids.cuda()

    with torch.inference_mode():
        output = pipe(sample['text'], max_length=input_ids.shape[1]+8)
        
    res.append(output[0]['generated_text'])

pred = []
labs = []
for i, r in enumerate(res):
    r = r.split('Answer:')[1]
    if 'true' in r.lower():
        pred.append('Entailment')
        labs.append(labels[i])
    elif 'false' in r.lower():
        pred.append('Contradiction')
        labs.append(labels[i])

result = compute_metrics_decoded(labs, pred)
print(result)


In [45]:
from sklearn.metrics import classification_report

print(classification_report(labs, pred))

               precision    recall  f1-score   support

Contradiction       0.71      0.65      0.68        95
   Entailment       0.66      0.72      0.69        90

     accuracy                           0.69       185
    macro avg       0.69      0.69      0.69       185
 weighted avg       0.69      0.69      0.69       185



In [44]:
result

{'micro_f1': 0.6864864864864865,
 'macro_f1': 0.6864040215104045,
 'precision': 0.6864864864864865,
 'recall': 0.6864864864864865,
 'accuracy': 0.6864864864864865}

Llama2-7b

In [7]:
from transformers import AutoTokenizer, AutoModelForCausalLM
from transformers import LlamaTokenizer, LlamaForCausalLM, pipeline

tokenizer = AutoTokenizer.from_pretrained("models/llama-2-7b")
tokenizer.pad_token = tokenizer.eos_token  # Most LLMs don't have a pad token by default
model = AutoModelForCausalLM.from_pretrained(
    "models/llama-2-7b", device_map="auto",# load_in_4bit=True
)





/usr/local/lib/python3.10/dist-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Loading checkpoint shards: 100%|██████████| 2/2 [00:34<00:00, 17.40s/it]


In [17]:
from transformers import LlamaForCausalLM, LlamaTokenizer

tokenizer = LlamaTokenizer.from_pretrained("models/llama-2-7b")
tokenizer.pad_token = tokenizer.eos_token
model = LlamaForCausalLM.from_pretrained("models/llama-2-7b", device_map="auto")

Loading checkpoint shards: 100%|██████████| 2/2 [00:13<00:00,  6.87s/it]


In [18]:
pipe = pipeline(
    task='text-generation',
    model=model,
    tokenizer=tokenizer,
    #max_new_tokens=64,
    #max_length=2048,
    do_sample=True,
    temperature=0.7,
    top_p=0.95,
    top_k=40,
    repetition_penalty=1.1,
  
)

In [86]:
# BEST

PROMPT = """{premise}
###Question: Does this imply that {hypothesis} ? Answer Yes or No
###Answer:
"""
text = "<s>[INST] What is your favourite condiment? [/INST]"
"Well, I'm quite partial to a good squeeze of fresh lemon juice. It adds just the right amount of zesty flavour to whatever I'm cooking up in the kitchen!</s> "
"[INST] Do you have mayonnaise recipes? [/INST]"


PROMPT_1 = """<s>[INST]Context
Premise: {premise}
Hypothesis: {hypothesis}
If Premise entails Hypothesis then return Entailment otherwise return Contradiction[/INST]
Output: 
"""

PROMPT2 = """
<s>[INST] 
<<SYS>>
You are a helpful, respectful and honest assistant. Don't provide extra explanation.
<</SYS>>

{premise} 
###Question: Does this imply that {hypothesis} ? No or Yes[/INST]

###Answer:
"""

samples = []
for sample in data_expanded:
    primary_evidence = "".join(sample['primary_evidence'])
    sentence = f"Primary trial evidence are {primary_evidence}"
    secondary_evidence = sample.get("secondary_evidence")
    if secondary_evidence:
        secondary_evidence = "".join(sample['secondary_evidence'])
        sentence = f"{sentence} Secondary trial evidence are {secondary_evidence}"
    input_text = get_input_text_p(sentence, sample['statement'], PROMPT2)
    temp = {"text":input_text, "label":sample['label']}
    samples.append(temp)

res = []
labels = []
for sample in tqdm(samples):
    labels.append(sample['label'])
    input_ids = tokenizer(
            sample['text'], return_tensors="pt"
        ).input_ids.cuda()

    with torch.inference_mode():
        output = pipe(sample['text'], max_length=input_ids.shape[1]+8)
        
    res.append(output[0]['generated_text'])

pred = []
labs = []
for i, r in enumerate(res):
    r = r.split('Answer:')[1]
    if 'yes' in r.lower():
        pred.append('Entailment')
        labs.append(labels[i])
    elif 'no' in r.lower():
        pred.append('Contradiction')
        labs.append(labels[i])

result = compute_metrics_decoded(labs, pred)
print(result)



  0%|          | 0/200 [00:00<?, ?it/s]/usr/local/lib/python3.10/dist-packages/transformers/pipelines/base.py:1101: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
 91%|█████████ | 182/200 [00:55<00:06,  2.81it/s]Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
100%|██████████| 200/200 [01:00<00:00,  3.29it/s]

{'micro_f1': 0.5114942528735632, 'macro_f1': 0.5087521174477696, 'precision': 0.5114942528735632, 'recall': 0.5114942528735632, 'accuracy': 0.5114942528735632}


In [87]:
from sklearn.metrics import classification_report

print(classification_report(labs, pred))

               precision    recall  f1-score   support

Contradiction       0.52      0.57      0.55        89
   Entailment       0.50      0.45      0.47        85

     accuracy                           0.51       174
    macro avg       0.51      0.51      0.51       174
 weighted avg       0.51      0.51      0.51       174



Llama2 13B

In [6]:
from transformers import LlamaForCausalLM, LlamaTokenizer

tokenizer = LlamaTokenizer.from_pretrained("models/llama-2-13b")
tokenizer.pad_token = tokenizer.eos_token
model = LlamaForCausalLM.from_pretrained("models/llama-2-13b", device_map="auto")

/usr/local/lib/python3.10/dist-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
You are using the default legacy behaviour of the <class 'transformers.models.llama.tokenization_llama.LlamaTokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565
Loading checkpoint shards: 100%|██████████| 3/3 [00:25<00:00,  8.63s/it]


In [9]:
from transformers import pipeline
pipe = pipeline(
    task='text-generation',
    model=model,
    tokenizer=tokenizer,
    #max_new_tokens=64,
    #max_length=2048,
    do_sample=True,
    temperature=0.7,
    top_p=0.95,
    top_k=40,
    repetition_penalty=1.1,
  
)

In [16]:
# BEST

PROMPT = """{premise}
###Question: Does this imply that {hypothesis} ? Answer Yes or No
###Answer:
"""
text = "<s>[INST] What is your favourite condiment? [/INST]"
"Well, I'm quite partial to a good squeeze of fresh lemon juice. It adds just the right amount of zesty flavour to whatever I'm cooking up in the kitchen!</s> "
"[INST] Do you have mayonnaise recipes? [/INST]"


PROMPT1 = """<s>[INST]Context
Premise: {premise}
Hypothesis: {hypothesis}
If Premise entails Hypothesis then return Entailment otherwise return Contradiction[/INST]
Output: 
"""

PROMPT2 = """
<s>[INST] 
<<SYS>>
You are a helpful, respectful and honest assistant. Don't provide extra explanation.
<</SYS>>

{premise} 
###Question: Does this imply that {hypothesis} ? No or Yes[/INST]

###Answer:
"""

samples = []
for sample in data_expanded:
    primary_evidence = "".join(sample['primary_evidence'])
    sentence = f"Primary trial evidence are {primary_evidence}"
    secondary_evidence = sample.get("secondary_evidence")
    if secondary_evidence:
        secondary_evidence = "".join(sample['secondary_evidence'])
        sentence = f"{sentence} Secondary trial evidence are {secondary_evidence}"
    input_text = get_input_text_p(sentence, sample['statement'], PROMPT2)
    temp = {"text":input_text, "label":sample['label']}
    samples.append(temp)

res = []
labels = []
for sample in tqdm(samples):
    labels.append(sample['label'])
    input_ids = tokenizer(
            sample['text'], return_tensors="pt"
        ).input_ids.cuda()

    with torch.inference_mode():
        output = pipe(sample['text'], max_length=input_ids.shape[1]+8)
        
    res.append(output[0]['generated_text'])

pred = []
labs = []
for i, r in enumerate(res):
    r = r.split('Answer:')[1]
    if 'yes' in r.lower():
        pred.append('Entailment')
        labs.append(labels[i])
    elif 'no' in r.lower():
        pred.append('Contradiction')
        labs.append(labels[i])

result = compute_metrics_decoded(labs, pred)
print(result)



  0%|          | 0/200 [00:00<?, ?it/s]/usr/local/lib/python3.10/dist-packages/transformers/pipelines/base.py:1101: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
100%|██████████| 200/200 [01:41<00:00,  1.96it/s]

{'micro_f1': 0.46022727272727276, 'macro_f1': 0.4380020839635642, 'precision': 0.4602272727272727, 'recall': 0.4602272727272727, 'accuracy': 0.4602272727272727}


In [17]:
from sklearn.metrics import classification_report

print(classification_report(labs, pred))

               precision    recall  f1-score   support

Contradiction       0.46      0.67      0.55        86
   Entailment       0.45      0.26      0.33        90

     accuracy                           0.46       176
    macro avg       0.46      0.46      0.44       176
 weighted avg       0.46      0.46      0.44       176



Llama2-70B

In [9]:
!pip install optimum auto-gptq 

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


Looking in indexes: http://pypi-cache/index, https://pypi.ngc.nvidia.com
You should consider upgrading via the '/usr/bin/python -m pip install --upgrade pip' command.


In [7]:
from transformers import AutoTokenizer, AutoModelForCausalLM
from transformers import LlamaTokenizer, LlamaForCausalLM, pipeline
tokenizer = AutoTokenizer.from_pretrained("models/TheBloke_Llama-2-70B-GPTQ_gptq-4bit-32g-actorder_True")
tokenizer.pad_token = tokenizer.eos_token
model = AutoModelForCausalLM.from_pretrained("models/TheBloke_Llama-2-70B-GPTQ_gptq-4bit-32g-actorder_True", device_map='auto')

pipe = pipeline(
    task='text-generation',
    model=model,
    tokenizer=tokenizer,
    #max_new_tokens=64,
    do_sample=True,
    temperature=0.7,
    top_p=0.95,
    top_k=40,
    repetition_penalty=1.1
)

CUDA extension not installed.
CUDA extension not installed.


In [10]:
# BEST

PROMPT = """{premise}
###Question: Does this imply that {hypothesis} ? Answer Yes or No
###Answer:
"""
text = "<s>[INST] What is your favourite condiment? [/INST]"
"Well, I'm quite partial to a good squeeze of fresh lemon juice. It adds just the right amount of zesty flavour to whatever I'm cooking up in the kitchen!</s> "
"[INST] Do you have mayonnaise recipes? [/INST]"


PROMPT1 = """<s>[INST]Context
Premise: {premise}
Hypothesis: {hypothesis}
If Premise entails Hypothesis then return Entailment otherwise return Contradiction[/INST]
Output: 
"""

PROMPT2 = """
<s>[INST] 
<<SYS>>
You are a helpful, respectful and honest assistant. Don't provide extra explanation.
<</SYS>>

{premise} 
###Question: Does this imply that {hypothesis} ? No or Yes[/INST]

###Answer:
"""

samples = []
for sample in data_expanded:
    primary_evidence = "".join(sample['primary_evidence'])
    sentence = f"Primary trial evidence are {primary_evidence}"
    secondary_evidence = sample.get("secondary_evidence")
    if secondary_evidence:
        secondary_evidence = "".join(sample['secondary_evidence'])
        sentence = f"{sentence} Secondary trial evidence are {secondary_evidence}"
    input_text = get_input_text_p(sentence, sample['statement'], PROMPT2)
    temp = {"text":input_text, "label":sample['label']}
    samples.append(temp)

res = []
labels = []
for sample in tqdm(samples):
    labels.append(sample['label'])
    input_ids = tokenizer(
            sample['text'], return_tensors="pt"
        ).input_ids.cuda()

    with torch.inference_mode():
        output = pipe(sample['text'], max_length=input_ids.shape[1]+8)
        
    res.append(output[0]['generated_text'])

pred = []
labs = []
for i, r in enumerate(res):
    r = r.split('Answer:')[1]
    if 'yes' in r.lower():
        pred.append('Entailment')
        labs.append(labels[i])
    elif 'no' in r.lower():
        pred.append('Contradiction')
        labs.append(labels[i])

result = compute_metrics_decoded(labs, pred)
print(result)



  3%|▎         | 6/200 [01:56<1:02:57, 19.47s/it]/usr/local/lib/python3.10/dist-packages/transformers/pipelines/base.py:1101: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(
100%|██████████| 200/200 [1:01:16<00:00, 18.38s/it]

{'micro_f1': 0.5555555555555556, 'macro_f1': 0.5555311676909571, 'precision': 0.5555555555555556, 'recall': 0.5555555555555556, 'accuracy': 0.5555555555555556}


In [11]:
from sklearn.metrics import classification_report

print(classification_report(labs, pred))

               precision    recall  f1-score   support

Contradiction       0.59      0.52      0.55        71
   Entailment       0.53      0.59      0.56        64

     accuracy                           0.56       135
    macro avg       0.56      0.56      0.56       135
 weighted avg       0.56      0.56      0.56       135



Llemma-7B

In [6]:
from transformers import AutoTokenizer, AutoModelForCausalLM
from transformers import LlamaTokenizer, LlamaForCausalLM, pipeline
tokenizer = AutoTokenizer.from_pretrained("models/llemma_7b")
tokenizer.pad_token = tokenizer.eos_token
model = AutoModelForCausalLM.from_pretrained("models/llemma_7b", device_map='auto')

pipe = pipeline(
    task='text-generation',
    model=model,
    tokenizer=tokenizer,
    #max_new_tokens=64,
    do_sample=True,
    temperature=0.7,
    top_p=0.95,
    top_k=40,
    repetition_penalty=1.1
)

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

/usr/local/lib/python3.10/dist-packages/torch/_utils.py:783: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()


In [39]:
# BEST

PROMPT = """{premise}
###Question: Does this imply that {hypothesis} ? Answer Yes or No
###Answer:
"""
text = "<s>[INST] What is your favourite condiment? [/INST]"
"Well, I'm quite partial to a good squeeze of fresh lemon juice. It adds just the right amount of zesty flavour to whatever I'm cooking up in the kitchen!</s> "
"[INST] Do you have mayonnaise recipes? [/INST]"


PROMPT1 = """<s>
Input: {premise}. Does this imply that {hypothesis}? Answer Yes or No
Response: 
"""

PROMPT2 = """
<s>[INST] 
<<SYS>>
You are a helpful, respectful and honest assistant. Don't provide extra explanation.
<</SYS>>

{premise} 
###Question: Does this imply that {hypothesis} ? No or Yes[/INST]

###Answer:
"""

PROMPT3 = """
Below is an instruction that describes a task, paired with an input that provides further context.Write a response that appropriately completes the request.\n\n"
"### Instruction:\nRead Premise and Hypothesis. If Hypothesis can be entailed from Premise repond with Entailment. If hypothesis cannot be entailed then repond with Contradiction.\n\n
### Input:\nPremise:{premise} Hypothesis:{hypothesis}\n\n### Response:"
"""

samples = []
for sample in data_expanded:
    primary_evidence = "".join(sample['primary_evidence'])
    sentence = f"Primary trial evidence are {primary_evidence}"
    secondary_evidence = sample.get("secondary_evidence")
    if secondary_evidence:
        secondary_evidence = "".join(sample['secondary_evidence'])
        sentence = f"{sentence} Secondary trial evidence are {secondary_evidence}"
    input_text = get_input_text_p(sentence, sample['statement'], PROMPT1)
    temp = {"text":input_text, "label":sample['label']}
    samples.append(temp)


In [44]:
sample = samples[8]
input_ids = tokenizer(
            sample['text'], return_tensors="pt"
        ).input_ids.cuda()

with torch.inference_mode():
    output = pipe(sample['text'], max_length=input_ids.shape[1]+8)
    
output[0]['generated_text']

'<s>\nInput: Primary trial evidence are Outcome Measurement:   Progression-free Survival (PFS) and to Evaluate Safety of the Trastuzumab, Bevacizumab and Docetaxel Regimen.  The trial was designed as a single-stage phase II rather then usual two-stage design because of the progression free survival (PFS) primary endpoint, as it is impractical to wait to assess PFS for patients in the first stage. We will consider a PFS of 50% at twelve months (median PFS of 12 months) or less uninteresting and a PFS of 70% at twelve months (median PFS of twenty months) worthy of pursuing the regimen in a future trials. The single-stage design is as follows: p0=0.50, p1=0.70, α=0.10, β= 0.10. This leads to a total sample size of 39 patients, 24 or higher of who are progression-free at 12 months.  Time frame: up to 3 yearsResults 1:   Arm/Group Title: Trastuzumab, Bevacizumab, and Docetaxel  Arm/Group Description: Trastuzumab g/kg], Bevacizumab g/kg], and Docetaxel [75 mg/M ]  Overall Number of Participa

In [55]:
# BEST

PROMPT = """{premise}
###Question: Does this imply that {hypothesis} ? Answer Yes or No
###Answer:
"""
text = "<s>[INST] What is your favourite condiment? [/INST]"
"Well, I'm quite partial to a good squeeze of fresh lemon juice. It adds just the right amount of zesty flavour to whatever I'm cooking up in the kitchen!</s> "
"[INST] Do you have mayonnaise recipes? [/INST]"


PROMPT1 = """<s>
Input: {premise}. Does this imply that {hypothesis}? Answer Yes or No
Response: 
"""

PROMPT2 = """
<s>[INST] 
<<SYS>>
You are a helpful, respectful and honest assistant. Don't provide extra explanation.
<</SYS>>

{premise} 
###Question: Does this imply that {hypothesis} ? No or Yes[/INST]

###Answer:
"""

PROMPT3 = """
Below is an instruction that describes a task, paired with an input that provides further context.Write a response that appropriately completes the request.\n\n"
"### Instruction:\nRead Premise and Hypothesis. If Hypothesis can be entailed from Premise repond with Entailment. If hypothesis cannot be entailed then repond with Contradiction.\n\n
### Input:\nPremise:{premise} Hypothesis:{hypothesis}\n\n### Response:"
"""

samples = []
for sample in data_expanded:
    primary_evidence = "".join(sample['primary_evidence'])
    sentence = f"Primary trial evidence are {primary_evidence}"
    secondary_evidence = sample.get("secondary_evidence")
    if secondary_evidence:
        secondary_evidence = "".join(sample['secondary_evidence'])
        sentence = f"{sentence} Secondary trial evidence are {secondary_evidence}"
    input_text = get_input_text_p(sentence, sample['statement'], PROMPT3)
    temp = {"text":input_text, "label":sample['label']}
    samples.append(temp)

res = []
labels = []
for sample in tqdm(samples):
    labels.append(sample['label'])
    input_ids = tokenizer(
            sample['text'], return_tensors="pt"
        ).input_ids.cuda()
    ans = False
    i= 0
    while ans == False:
        with torch.inference_mode():
            output = pipe(sample['text'], max_length=input_ids.shape[1]+8)
            resp = output[0]['generated_text'].split('Response:')[1].lower()
            i+=1
            if 'entailment' in resp or 'contradiction' in resp or i>10:
                ans=True
        
    res.append(output[0]['generated_text'])

pred = []
labs = []
for i, r in enumerate(res):
    r = r.split('Response:')[1]
    if 'entailment' in r.lower():
        pred.append('Entailment')
        labs.append(labels[i])
    elif 'contradiction' in r.lower():
        pred.append('Contradiction')
        labs.append(labels[i])

result = compute_metrics_decoded(labs, pred)
print(result)



100%|██████████| 200/200 [01:46<00:00,  1.87it/s]

{'micro_f1': 0.5353535353535354, 'macro_f1': 0.515583448569301, 'precision': 0.5353535353535354, 'recall': 0.5353535353535354, 'accuracy': 0.5353535353535354}


In [56]:
from sklearn.metrics import classification_report

print(classification_report(labs, pred))

               precision    recall  f1-score   support

Contradiction       0.56      0.33      0.42        99
   Entailment       0.53      0.74      0.61        99

     accuracy                           0.54       198
    macro avg       0.54      0.54      0.52       198
 weighted avg       0.54      0.54      0.52       198



In [53]:
from sklearn.metrics import classification_report

print(classification_report(labs, pred))

               precision    recall  f1-score   support

Contradiction       0.53      0.73      0.61       100
   Entailment       0.56      0.34      0.42       100

     accuracy                           0.54       200
    macro avg       0.54      0.54      0.52       200
 weighted avg       0.54      0.54      0.52       200



In [51]:
from sklearn.metrics import classification_report

print(classification_report(labs, pred))

               precision    recall  f1-score   support

Contradiction       0.51      0.50      0.51        94
   Entailment       0.51      0.52      0.51        93

     accuracy                           0.51       187
    macro avg       0.51      0.51      0.51       187
 weighted avg       0.51      0.51      0.51       187



In [47]:
from sklearn.metrics import classification_report

print(classification_report(labs, pred))

               precision    recall  f1-score   support

Contradiction       0.52      0.76      0.61        98
   Entailment       0.54      0.29      0.38        97

     accuracy                           0.52       195
    macro avg       0.53      0.52      0.49       195
 weighted avg       0.53      0.52      0.50       195

